# <center> Определение тональности мелодии

### Цель работы, или зачем оно все нам нужно

Тональность песни, ипользуется в качестве признака при построении рекомендательной системы для существующих аудиотреков и автоматического аннонтировании новых, где нужно большое количество признаков, построенных на мелодиии. А также как метафича для других алгоритмов машинного обучения.
А ещё: <img src="../../img/why_not.jpeg">

 ### Кратко о требуемой музыкальной теории 

**Определение:** Тональность – это закрепление положения музыкального лада за определёнными по высоте звучания музыкальными тонами, привязка к конкретному участку музыкального звукоряда. 

Сложно, да? Разберемся подробнее.

Любая тональность состоит из двух состовляющих: тоники и лада
<font color="blue"><center>**тональность = тоника + лад**</font>
    
Лад – это система звуков, объединенных устойчивым центром – тоникой.
Лады состоят из семи звуков. Ступени (ноты) лада принято обозначать римскими цифрами.
Первая ступень, тоника – самый устойчивый звук лада.

Два основных семиступенных лада – мажорный (мажор) и минорный (минор).  По характеру мажор сравнивают со светом, минор – с тенью. «Мажор» в переводе с итальянского языка означает больший, «минор» – меньший. 

**Мажор** – это такой лад, в котором на тонике образуется мажорное трезвучие. Оно называется тоническое трезвучие. 

**Минор** – это такой лад, в котором на котором образуется минорное трезвучие. Оно также называется тоническим.

Тоническое трезвучие состоит из: I – III – V ступеней. Расстояние между соседними звуками в мажорном и минорном трезвучие разное. Мажор: 2 тона – 1,5 тона; минор:  1,5 тона – 2 тона.

Подведем итог: если говорить грубо, то тоника, это ступень(или нота), вокруг которой строится вся мелодия, то есть центральная. А лад, в свою очередь, определяет настроение музыкальной композиции.

Итак у нас есть 7 нот:
- До (C);
- Ре (D);
- Ми (E);
- Фа (F);
- Соль (G);
- Ля (A);
- Си (B);

которые могут быть тоникой, а так же 5 нот увеличенные на полтона (C#, D#, F#, G#, A#). И два вида ладов: мажор (major) и минор (minor). 
А значит тональностей у нас 24.
<center>
<img src="../../img/notes.jpg">

## Постановка задачи

Исходя из всего выше сказанного, вопрос определения тональности сводтся к задаче классиффикации на 24 класса.


## Данные

В проекте мы используем 3 датасета:
- **[GTZAN](http://visal.cs.cityu.edu.hk/downloads/gtzan-keys/)** 
Датасет стоит из 1000 музыкальных композиций по 30 секунд, данные размечены и собраны в файл desription_GTZAN.csv
Разделим его на три части: 62,5% обучающая выборка + 12,5% валидационная GT_TV, 25% тестовая выборка GT_TE.
- **[GiantSteps Key Dataset](https://github.com/GiantSteps/giantsteps-key-dataset)**
Датасет стоит из 603 музыкальных композиций по 2 минуты, данные размечены и собраны в файл desription_GS.csv
Эти данные будем использовать только как тестовые.
- **[GiantSteps MTG Key Dataset](https://github.com/GiantSteps/giantsteps-mtg-key-dataset)**
Датасет стоит из 1486 музыкальных композиций по 2 минуты, данные размечены и собраны в файл desription_GS_MTG.csv
Этот датасет исспользуем как тренировойчной и валидационный 

Во всех трех датасетах изначально песни представленны в формате '.mp3'. Конвертируем эти файлы в формат '.wav', используя баш-скрипт convert_dl.sh
После того как провели все манипуляции собираем все данные в [data](https://yadi.sk/d/piIGdyig3Uf6eQ)

In [ ]:
from __future__ import print_function
import glob, os
import pandas as pd
import numpy as np
import pickle

In [ ]:
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

import IPython.display

import madmom

import librosa
import librosa.display

In [ ]:
from tqdm import tqdm
import wave
from scipy.io import wavfile
SAMPLE_RATE = 44100

import seaborn as sns
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()
import plotly.tools as tls

# Math
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile

from sklearn.model_selection import train_test_split

### GiantSteps Key Dataset

Теперь посмотрим, что у нас внутри датасетов.

In [ ]:
PATH = '../data/giantsteps-key-dataset-master/'
description_GS = pd.read_csv(PATH + 'description_master.csv')

In [ ]:
description_GS.head()

Как мы можем заметить, ничего лишнего. Только название файла и тональность.

Теперь посмотрим как распределены данные по тональностям. 

In [ ]:
print("Total number of labels in training data : ",len(description_GS['Target'].value_counts()))
print("Labels are : ", description_GS['Target'].unique())
plt.figure(figsize=(15,8))
audio_type = description_GS['Target'].value_counts()
sns.barplot(audio_type.values, audio_type.index, palette=sns.color_palette("husl", 24))
for i, v in enumerate(audio_type.values):
    plt.text(0.8,i,v,color='k',fontsize=12)
plt.xticks(rotation='vertical')
plt.xlabel('Frequency')
plt.ylabel('Label Name')
plt.title("Labels with their frequencies in training data")
plt.show()

## GiantSteps MTG Key Dataset

In [ ]:
PATH = '../data/giantsteps-mtg-key-dataset-master/'
description_GS_mtg = pd.read_csv(PATH + 'description_master.csv')

In [ ]:
description_GS_mtg.head()

In [ ]:
print("Total number of labels in training data : ",len(description_GS_mtg['Target'].value_counts()))
print("Labels are : ", description_GS_mtg['Target'].unique())
plt.figure(figsize=(15,8))
audio_type = description_GS_mtg['Target'].value_counts()
sns.barplot(audio_type.values, audio_type.index, palette=sns.color_palette("husl", 24))
for i, v in enumerate(audio_type.values):
    plt.text(0.8,i,v,color='k',fontsize=12)
plt.xticks(rotation='vertical')
plt.xlabel('Frequency')
plt.ylabel('Label Name')
plt.title("Labels with their frequencies in training data")
plt.show()

Так-так. А тут у нас затесались нелегалы. Удаляем все записи у которых не определенна тональность.

In [ ]:
description_GS_mtg.drop(index = description_GS_mtg[description_GS_mtg['Target'] =='None'].index,
                                             axis=0, inplace=True)

In [ ]:
print("Total number of labels in training data : ",len(description_GS_mtg['Target'].value_counts()))
print("Labels are : ", description_GS_mtg['Target'].unique())
plt.figure(figsize=(15,8))
audio_type = description_GS_mtg['Target'].value_counts()
sns.barplot(audio_type.values, audio_type.index, palette=sns.color_palette("husl", 24))
for i, v in enumerate(audio_type.values):
    plt.text(0.8,i,v,color='k',fontsize=12)
plt.xticks(rotation='vertical')
plt.xlabel('Frequency')
plt.ylabel('Label Name')
plt.title("Labels with their frequencies in training data")
plt.show()

Славненько. Поехали дельше

### GTZAN

In [ ]:
description_GTZAN = pd.read_csv("../data/GTZAN/description.csv")

In [ ]:
description_GTZAN.head()

In [ ]:
print("Total number of labels in training data : ",len(description_GTZAN['Target'].value_counts()))
print("Labels are : ", description_GTZAN['Target'].unique())
plt.figure(figsize=(15,8))
audio_type = description_GTZAN['Target'].value_counts()
sns.barplot(audio_type.values, audio_type.index, palette=sns.color_palette("husl", 24))
for i, v in enumerate(audio_type.values):
    plt.text(0.8,i,v,color='k',fontsize=12)
plt.xticks(rotation='vertical')
plt.xlabel('Frequency')
plt.ylabel('Label Name')
plt.title("Labels with their frequencies in training data")
plt.show()

In [ ]:
description_GTZAN.drop(index = description_GTZAN[description_GTZAN['Target'] =='None'].index,
                                             axis=0, inplace=True)

In [ ]:
print("Total number of labels in training data : ",len(description_GTZAN['Target'].value_counts()))
print("Labels are : ", description_GTZAN['Target'].unique())
plt.figure(figsize=(15,8))
audio_type = description_GTZAN['Target'].value_counts()
sns.barplot(audio_type.values, audio_type.index, palette=sns.color_palette("husl", 24))
for i, v in enumerate(audio_type.values):
    plt.text(0.8,i,v,color='k',fontsize=12)
plt.xticks(rotation='vertical')
plt.xlabel('Frequency')
plt.ylabel('Label Name')
plt.title("Labels with their frequencies in training data")
plt.show()

## Предобработка данных

Посмотрим что же мы можем сделать с музыкой.

Возьмем файл 32 из датасета GS_mtg


In [ ]:
path_file = '../data/giantsteps-mtg-key-dataset-master/audio/' + description_GS_mtg.Name[32] + '.wav'
y, sr = librosa.load(path_file, duration=30)
plt.figure(figsize=(10, 4))
librosa.display.waveplot(y, sr=sr)
plt.tight_layout()

Как мы видим из графика (а так же знаем еще со школы) звук - волна. Только в таком виде что-либо делать с ней   проблематично.
Посмотрим на частотно-временное представление волны, то есть спектрограмму.

**Спектрограмма** — изображение, показывающее зависимость спектральной плотности мощности сигнала от времени.

Спектрограмма обычно создаются одним из двух способов: аппроксимируется, как набор фильтров, полученных из серии полосовых фильтров, или рассчитывается по сигналу времени, используя оконное преобразование Фурье.

Создание спектрограммы с помощью оконного преобразования Фурье обычно выполняется методами цифровой обработки. Производится цифровая выборка данных во временной области. Сигнал разбивается на части, которые, как правило, перекрываются, и затем производится преобразование Фурье, чтобы рассчитать величину частотного спектра для каждой части. Каждая часть соответствует вертикальной линии на изображении — значение амплитуды в зависимости от частоты в каждый момент времени. Спектры или временные графики располагаются рядом на изображении или трёхмерной диаграмме.

Оконное преобразование Фурье — это разновидность преобразования Фурье, определяемая следующим образом:

$$F(t,\omega) = \int\limits_{-\infty}^\infty f(\tau) W(\tau-t) e^{-i\omega \tau}\,d\tau$$

где $W(τ − t)$ — некоторая оконная функция. 

В случае дискретного преобразования оконная функция используется аналогично:

$$F(m,\omega) = \sum_{n=-\infty}^{\infty} f[n]w[n-m]e^{-j \omega n} $$

На основе спектра строится функция цветности. Хромограмма - интересное и мощное представление для музыкального аудио, в котором весь спектр проецируется на 12 ячеек, представляющих 12 отдельных полутонов (или цветности) музыкальной октавы. Поскольку в музыке ноты, расположенные на на растоянии в октаву (т.е 6 тонов) друг от друга, воспринимаются как особенно похожие, зная, что распределение цветности даже без абсолютной частоты (т.е. исходной октавы) может дать полезную музыкальную информацию об аудио - и может даже показать воспринимаемое музыкальное сходство, которое не проявляется в исходных спектрах.

In [ ]:
chroma = librosa.feature.chroma_stft(y, sr = sr, n_fft=5000)
plt.figure(figsize=(13, 4))
librosa.display.specshow(chroma, y_axis='chroma', x_axis='time')
plt.colorbar()
plt.title('Chromagram')
plt.tight_layout()

Из хромограммы уже просматриваются особенности изманения с течением времени нот.

Исходя из определения, что тоника - то самый устойчивая нота и вокруг нее крутится вся мелодия. Можно посмотреть ноту у которой будет наибольшее значение средней по времени хромограммы. 

In [ ]:
chroma.mean(axis = 1).argmax()

9 - это нота Ля(A). Посмотрим на реальное значение. 

In [ ]:
description_GS_mtg.Target[32]

Юху. Они совпали. Имеет смысл предположить, что вычесленной функции цветности будет достаточно что бы определить тональность музыкальной композиции. 

Расчитаем для всех наших данных хромограммы.

In [ ]:
n1 = description_GTZAN.shape[0]
n2 = 12 # количество различных нот
n3 = 431 * 2# за 10 секунд делается 431 кодр, а мы берем первые 20 секунд

GTZAN_chroma = np.empty((n1, n2, n3))
i = 0
pathForData = '../data/GTZAN/'

for file in tqdm(description_GTZAN.Name):
    audio_path = pathForData + 'all/' + file + '.wav'
    y, sr = librosa.load(audio_path, duration=20)
    chroma = librosa.feature.chroma_stft(y, sr = sr, n_fft=5000)
    GTZAN_chroma[i] = chroma
    i += 1
with open(pathForData + 'GTZAN_chromagram.txt', 'wb') as f:
    pickle.dump(GTZAN_chroma, f, 2)

In [ ]:
n1 = description_GS_mtg.shape[0]

GS_mtg_chroma = np.empty((n1, n2, n3))
i = 0
pathForData = '../data/giantsteps-mtg-key-dataset-master/'

for file in tqdm(description_GS_mtg.Name):
    audio_path = pathForData + 'audio/' + file + '.wav'
    y, sr = librosa.load(audio_path, duration=20)
    chroma = librosa.feature.chroma_stft(y, sr = sr, n_fft=5000)
    GS_mtg_chroma[i] = chroma
    i += 1
with open(pathForData + 'GS_mtg_chromagram.txt', 'wb') as f:
    pickle.dump(GS_mtg_chroma, f, 2)

In [ ]:
n1 = description_GS.shape[0]

GS_chroma = np.empty((n1, n2, n3))
i = 0
pathForData = '../data/giantsteps-key-dataset-master/'


for file in tqdm(description_GS.Name):
    audio_path = pathForData + 'audio/' + file + '.wav'
    y, sr = librosa.load(audio_path, duration=20)
    chroma = librosa.feature.chroma_stft(y, sr = sr, n_fft=5000)
    GS_chroma[i] = chroma
    i += 1
with open(pathForData + 'GS_chromagram.txt', 'wb') as f:
    pickle.dump(GS_chroma, f, 2)

## Модель

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import SGD, Adagrad
from __future__ import division
import keras
import re
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import ELU, Softmax
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard    

In [ ]:
K.set_image_data_format('channels_first')

Ну что ж, настало время поговорить о модели, которая будет определять тональности.

Мы будем использавать сверточную нейронную сеть. 2 слоя сверток по 24 шаблона размерностью (12, 43), почему так?
Мы предпологаем, что каждая тональность имеет свой особенный шаблон, размер которого определяется как количество уникальных нот (по хромограмме которая подается на вход) на количество кадров в 1 секунду. При последовательном применении этих 2-х свертках наша нейроная сеть лучше определяет, усредняет эти шаблоны.

Далее мы усредняем по времени наши данные. После чего, исспользуя полносвязный слой, делим сначала на 48 выходов, для того что бы не потерять слишком много информации. А потом собственно определяем класс к которому относится объект.

На всех слоях, кроме последнего исспользуется функция активации 'ELU'. 
На последнем 'softmax'.

Вкачестве регуляризации на полносвязных слоях будем исспользовать dropout. Коэффициент будм подбирать на валидационной выборке. 

В качестве loss-функции будем исспользовать котегориальную крос-энтропию. А в качестве метрики acuracy.


In [ ]:
def get_model(coef_dropout):
    nclasses = 24 
    count_frame_one_sec = 43
    height = 12
    width = 431 * 2
    
    model = Sequential()

    model.add(Conv2D(nclasses, (height, count_frame_one_sec), 
                     padding='same',
                     input_shape=(1, height, width)))
    model.add(ELU())
    model.add(Conv2D(nclasses, (height, count_frame_one_sec), 
                     padding='same', 
                     input_shape=(1, height, width)))
    model.add(ELU())
    model.add(AveragePooling2D(pool_size = (1, width)))

    model.add(Flatten())

    model.add(Dense(48))
    model.add(ELU())
    model.add(Dropout(0.5))

    model.add(Dense(nclasses))
    model.add(Softmax())

    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    return model

In [ ]:
model = get_model(0.5)
SVG(model_to_dot(model, show_layer_names=True, show_shapes=True).create(prog='dot', format='svg'))

Теперь поговорим о метрике и оценках результатов.

В этом проекте используется метрика MIREX - это общепринятая метрика для данной задачи. В ней учитываюстя не только абсолютно верно классифицированные объекты, но и верные не по  всем критериям, а именно:

- Квинта(f). Это такой тип ошибки, когда верно определен лад, но тоника предказаная находится на расстоянии квинты(3.5 тона)  до реальной(или наоборот).
- Относительный мажор/минор(r): лад определен неверно и а) определен мажор и тоника, на 1.5 тона выше; б) определен минор, а тоника на 1.5 тона ниже реальной.
- Праллельный мажор/минор(p): лад определён не верно, а тоники совпадают

<center> **MIREX = right + 0.5f + 0.3r + 0.2p**</center>

In [ ]:
relative_key = {
     'C major': 'A minor'
    ,'G major': 'E minor'
    ,'F major': 'D minor'
    ,'D major': 'B minor'
    ,'A major': 'E minor'
    ,'E major': 'C# minor'
    ,'B major': 'G# minor'
    ,'F# major': 'D# minor'
    ,'C# major': 'A# minor'
    ,'G# major': 'F minor'
    ,'D# major': 'C minor'
    ,'A# major': 'G minor'
    
    #other same, but reverse
    ,'A minor': 'C major'
    ,'E minor': 'G major'
    ,'D minor': 'F major'
    ,'B minor': 'D major'
    ,'E minor': 'A major'
    ,'C# minor': 'E major'
    ,'G# minor': 'B major'
    ,'D# minor': 'F# major'
    ,'A# minor': 'C# major'
    ,'F minor': 'G# major'
    ,'C minor': 'D# major'
    ,'G minor': 'A# major'
    
}
fifth_key = {
    'C': 'G',
    'C#': 'G#',
    'D': 'A',
    'D#': 'A#',
    'E': 'B',
    'F': 'C',
    'F#': 'C#',
    'G': 'D',
    'G#': 'D',
    'A': 'E',
    'A#': 'F',
    'B': 'F#'
    
}
col = pd.get_dummies(description_GTZAN.Target).columns

In [ ]:
def SetMaxProbabilityToOne(array):
    return map(lambda x: 1 if(x == max(array)) else 0, array)

def GetIndexOneValue(array):
    for i, x in enumerate(array):
        if(x != 0):
            return i
        
def GetPredictNameAndYName(pred, y):
    pred = SetMaxProbabilityToOne(pred)
    
    indPred = GetIndexOneValue(pred)
    namePred = col[indPred]
    
    indY = GetIndexOneValue(y)
    nameY = col[indY]
    
    return namePred, nameY

def GetMIREXScoreOnSample(namePred, nameY):
    if(namePred == nameY):
        return 1
    
    isHaveRelativeKey = False
    try:
        isHaveRelativeKey = ((relative_key[namePred] == nameY) | (relative_key[nameY] == namePred))
    except:
        pass
    
    if(isHaveRelativeKey):
        return 0.3

    namePred = re.findall(r'\w+[#]*', namePred)
    nameY = re.findall(r'\w+[#]*', nameY)

    if(namePred[1] == nameY[1]) & ((fifth_key[namePred[0]] == nameY[0]) | (fifth_key[nameY[0]] == namePred[0])):

        return 0.5
    
    if(namePred[0] == nameY[0]):
        return 0.2
    
    return 0

def GetMIREXScore(model, X, y):
    score = 0.0
    correctPredict = 0
    correctGroundTruthKey = 0
    correctRelativeMajMinKey = 0
    correctParallelMajMinKey = 0
    other = 0
    for i in range(X.shape[0]):
        pred = model.predict(X[i].reshape(1, 1, X.shape[2], X.shape[3]))[0]
        yToFunc = y[i]
        namePred, nameY = GetPredictNameAndYName(pred, yToFunc)
        
        scoreOnCurrentSample = GetMIREXScoreOnSample(namePred, nameY)
        score += scoreOnCurrentSample
        
        if(scoreOnCurrentSample == 1):
            correctPredict += 1
        elif(scoreOnCurrentSample == 0.5):
            correctGroundTruthKey += 1
        elif(scoreOnCurrentSample == 0.3):
            correctRelativeMajMinKey += 1
        elif(scoreOnCurrentSample == 0.2):
            correctParallelMajMinKey += 1
        elif(scoreOnCurrentSample == 0):
            other += 1
        
    score /= X.shape[0]
    correctPredict 
    correctGroundTruthKey 
    correctRelativeMajMinKey 
    correctParallelMajMinKey 
    return score, X.shape[0], correctPredict, correctGroundTruthKey\
            , correctRelativeMajMinKey, correctParallelMajMinKey, other

Приведем данные к нужному виду

In [ ]:
GS_chroma.shape = (GS_chroma.shape[0], 1, n2, n3)
t_GS = pd.get_dummies(description_GS.Target)
y_GS = t_GS.as_matrix()

GS_mtg_chroma.shape = (GS_mtg_chroma.shape[0], 1, n2, n3)
t_GS_mtg = pd.get_dummies(description_GS_mtg.Target)
y_GS_mtg = t_GS_mtg.as_matrix()

GTZAN_chroma.shape = (GTZAN_chroma.shape[0], 1, n2, n3)
t_GTZAN = pd.get_dummies(description_GTZAN.Target)
y_GTZAN = t_GTZAN.as_matrix()

In [ ]:
X_train_GS, X_valid, y_train_GS, y_valid = train_test_split(GS_mtg_chroma, y_GS_mtg, test_size = 0.17, shuffle = True)

coef_dropout = np.linspace(0.75, 0.25, 5)

In [ ]:
res = {}
for coef in coef_dropout:
    X_train, X_test, y_train, y_test = train_test_split(X_valid, y_valid, test_size = 0.25, shuffle = True)
    model = get_model(coef)
    model.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0, 
              shuffle=True, verbose=0)
    score = GetMIREXScore(model, X_test, y_test)
    str_score = 'coef-' + str(coef)
    res[str_score] = score[0]
    K.clear_session()

### Обучение

In [ ]:
res

In [ ]:
model = get_model(0.75)
model.fit(X_train_GS, y_train_GS, batch_size=32, epochs=50, validation_split=0.1, 
          shuffle=True, verbose=1)

In [ ]:
score = GetMIREXScore(model, GS_chroma, y_GS)
print(score)
X_train, X_test, y_train, y_test = train_test_split(GTZAN_chroma, y_GTZAN, test_size = 0.25, shuffle = True)
score = GetMIREXScore(model, X_test, y_test)
print(score)

In [ ]:
K.clear_session()

In [ ]:
model1 = get_model(0.75)
model1.fit(X_train, y_train, batch_size=32, epochs=50, validation_split=0.1, 
          shuffle=True, verbose=1)

In [ ]:
score = GetMIREXScore(model1, X_test, y_test)
print(score)
score = GetMIREXScore(model1, GS_chroma, y_GS)
print(score)

In [ ]:
K.clear_session()

### Оценка полученых результатов.

Мы обучали нашу модель на двух разных датасетах (GeantStep_mtg и GTAZAN). И тестировали тоже на разных данных.
Что на выходе (оценка по mirex, количество объектов в тесте, количество верных определенных объектовб кол-во квинт, кол-во относительных, количество параллельных, др. ошибки)
Обучение на GS_mtg:
- тест GS: (0.6165837479270317, 603, 317, 77, 37, 26, 146)

- тест GTZAN: (0.6382775119617228, 209, 116, 21, 11, 18, 43)

Обучение на GTZAN:
- тест GS: (0.5688225538971808, 603, 278, 89, 43, 38, 155)
- тест GTZAN: (0.6507177033492826, 209, 116, 27, 9, 19, 38)

Чисто визуально кажется, что реезультат не велик, но мы можем заметить, что при рандоме наш результат был бы 0.04.
Во всех случаях определенно верно более половины объектов и менее 1/3 произвольных ошибок, что является хорошим показателем.

Из [cтатьи](https://arxiv.org/abs/1706.02921) вышедшей в 9.06.2017 лучшие результаты: 0.74

А значит нам есть куда еще расти.

### О дальнейших путях развития

- Первое и, возможно, самое важное для решения подобных задач: нужно больше данных;
- А когда много данных можно реализовать более сложную архетектуру нейронной сети;
- Так же можно подумать об внедрении в продакшн, так как задача остается актуальной и по сей день. 
Хотя [некоторые](http://www.jordipons.me/apps/music-audio-tagging-at-scale-demo/) уже представили работающий протатип.